1. Go through the BERT QA demo and run both parts of the demo on google colab (using 
BERT for QA directly and fine-tuning BERT for QA) (2 points) 
2. Fine-tune the BERT for question answering model on coqa dataset using a similar process 
as shown in the demo for the squad dataset (8 points) 
3. Import BERT for the classification model which is already fine-tuned and apply it to any text classification dataset such as the Twitter dataset. (5 points) 
4. Fine-tune the BERT for the classification model on the text classification dataset you used in the previous question and check the performance. (10 points) 
5. Report (5 pts)

In [1]:

!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 28.6 MB/s 
     |████████████████████████████████| 163 kB 61.4 MB/s 
     |████████████████████████████████| 7.6 MB 53.4 MB/s 


In [2]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 27.2 MB/s 
     |████████████████████████████████| 115 kB 57.9 MB/s 
     |████████████████████████████████| 95 kB 6.1 MB/s 
     |████████████████████████████████| 212 kB 76.0 MB/s 
     |████████████████████████████████| 127 kB 64.8 MB/s 
     |████████████████████████████████| 115 kB 62.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: dill
    Found existing installation: dill 0.3.6
    Uninstalling dill-0.3.6:
      Successfully uninstalled dill-0.3.6


In [4]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72 kB 1.4 MB/s 


In [6]:
# importing required libraries
import pandas as pd
import numpy as np
import torch
from transformers import BertForQuestionAnswering, BertForSequenceClassification,BertTokenizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline, AdamW, BertTokenizerFast, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from datasets import load_dataset
from transformers import get_scheduler

import evaluate
from tqdm import tqdm
from torch.utils.data import DataLoader

In [7]:
# loading the dataset
coqa = pd.read_json('http://downloads.cs.stanford.edu/nlp/data/coqa/coqa-train-v1.0.json')
coqa

,version,data
0,1,"{'source': 'wikipedia', 'id': '3zotghdk5ibi9ce..."
1,1,"{'source': 'cnn', 'id': '3wj1oxy92agboo5nlq4r7..."
2,1,"{'source': 'gutenberg', 'id': '3bdcf01ogxu7zdn..."
3,1,"{'source': 'cnn', 'id': '3ewijtffvo7wwchw6rtya..."
4,1,"{'source': 'gutenberg', 'id': '3urfvvm165iantk..."
...,...,...
7194,1,"{'source': 'gutenberg', 'id': '34j10vatjfyw0ao..."
7195,1,"{'source': 'cnn', 'id': '3vj40nv2qinjocrcy7k4z..."
7196,1,"{'source': 'race', 'id': '3rjsc4xj10uw0to3vq0v..."
7197,1,"{'source': 'wikipedia', 'id': '3gs6s824sqxty8v..."


In [8]:
# understanding the data
coqa["data"][0]

{'source': 'wikipedia',
 'id': '3zotghdk5ibi9cex97fepx7jetpso7',
 'filename': 'Vatican_Library.txt',
 'story': 'The Vatican Apostolic Library (), more commonly called the Vatican Library or simply the Vat, is the library of the Holy See, located in Vatican City. Formally established in 1475, although it is much older, it is one of the oldest libraries in the world and contains one of the most significant collections of historical texts. It has 75,000 codices from throughout history, as well as 1.1 million printed books, which include some 8,500 incunabula. \n\nThe Vatican Library is a research library for history, law, philosophy, science and theology. The Vatican Library is open to anyone who can document their qualifications and research needs. Photocopies for private study of pages from books published between 1801 and 1990 can be requested in person or by mail. \n\nIn March 2014, the Vatican Library began an initial four-year project of digitising its collection of manuscripts, to 

Here in the dataset we can see that there are 7200 rows, each row contains one paragraph and multiple question and answer pairs from that paragraph.

If we print the first row, we see that there are 20 questions and answers for the first paragraph and answers are in the form of start index and end index from the paragraph

this is the standard format of any closed domain question answering dataset

In [9]:
#dropping version column
del coqa["version"]


In [10]:
coqa

,data
0,"{'source': 'wikipedia', 'id': '3zotghdk5ibi9ce..."
1,"{'source': 'cnn', 'id': '3wj1oxy92agboo5nlq4r7..."
2,"{'source': 'gutenberg', 'id': '3bdcf01ogxu7zdn..."
3,"{'source': 'cnn', 'id': '3ewijtffvo7wwchw6rtya..."
4,"{'source': 'gutenberg', 'id': '3urfvvm165iantk..."
...,...
7194,"{'source': 'gutenberg', 'id': '34j10vatjfyw0ao..."
7195,"{'source': 'cnn', 'id': '3vj40nv2qinjocrcy7k4z..."
7196,"{'source': 'race', 'id': '3rjsc4xj10uw0to3vq0v..."
7197,"{'source': 'wikipedia', 'id': '3gs6s824sqxty8v..."


In [11]:
# structuring the dataset in usable format (creating one question and answer pair per row)
# by doing this, the text column will have repeated content until all questions of that paragraph are not covered
cols = ["text","question","answer"]
comp_list = []
for index, row in coqa.iterrows():
    for i in range(len(row["data"]["questions"])):
        temp_list = []
        temp_list.append(row["data"]["story"])
        temp_list.append(row["data"]["questions"][i]["input_text"])
        temp_list.append(row["data"]["answers"][i]["input_text"])
        comp_list.append(temp_list)
new_df = pd.DataFrame(comp_list, columns = cols) 
new_df

,text,question,answer
0,"The Vatican Apostolic Library (), more commonl...",When was the Vat formally opened?,It was formally established in 1475
1,"The Vatican Apostolic Library (), more commonl...",what is the library for?,research
2,"The Vatican Apostolic Library (), more commonl...",for what subjects?,"history, and law"
3,"The Vatican Apostolic Library (), more commonl...",and?,"philosophy, science and theology"
4,"The Vatican Apostolic Library (), more commonl...",what was started in 2014?,a project
...,...,...,...
108642,(CNN) -- Cristiano Ronaldo provided the perfec...,Who was a sub?,Xabi Alonso
108643,(CNN) -- Cristiano Ronaldo provided the perfec...,Was it his first game this year?,Yes
108644,(CNN) -- Cristiano Ronaldo provided the perfec...,What position did the team reach?,third
108645,(CNN) -- Cristiano Ronaldo provided the perfec...,Who was ahead of them?,Barca.


In [12]:
# Loading the BERT for question answering model which is already finetuned on squad and its corresponding tokenizer
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

Downloading:   0%|          | 0.00/443 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Here we also have to download the tokenizer because BERT follows a specific method for tokenizing words and hence with each type of BERT its corresponding tokenizer is required

In [13]:
# picking out a random question and answer pair from the dataset
random_num = np.random.randint(0,len(new_df))
question = new_df["question"][random_num]
text = new_df["text"][random_num]

In [14]:
# tokeninzing the text
input_ids = tokenizer.encode(question, text)
print("The input has a total of {} tokens.".format(len(input_ids)))

The input has a total of 471 tokens.


In [15]:
# visualizing the tokens
tokens = tokenizer.convert_ids_to_tokens(input_ids)
for token, id in zip(tokens, input_ids):
    print('{:8}{:8,}'.format(token,id))

[CLS]        101
what       2,054
does       2,515
the        1,996
legislation   6,094
promote    5,326
for        2,005
its        2,049
supporters   6,793
?          1,029
[SEP]        102
homosexual  15,667
couples    6,062
in         1,999
new        2,047
york       2,259
are        2,024
being      2,108
married    2,496
for        2,005
the        1,996
first      2,034
time       2,051
after      2,044
governor   3,099
andrew     4,080
cu        12,731
##omo     19,506
signed     2,772
a          1,037
law        2,375
permitting  24,523
same       2,168
-          1,011
sex        3,348
marriage   3,510
in         1,999
the        1,996
state      2,110
last       2,197
month      3,204
.          1,012
but        2,021
such       2,107
marriages  12,743
,          1,010
though     2,295
now        2,085
legal      3,423
in         1,999
new        2,047
york       2,259
,          1,010
remain     3,961
controversial   6,801
.          1,012
so         2,061
many       2,116

Here we see how each word is getting a unique token, and some rare words are getting split into multiple tokens. The token 101 is always the first token mentioning start of text and token 102 is the separator token which comes between the question and the answer and also at the end

In [16]:
# Visualizing the number of token in question and text
sep_idx = input_ids.index(tokenizer.sep_token_id)
print("SEP token index: ", sep_idx)
num_seg_a = sep_idx + 1
print("Number of tokens in segment A (question): ", num_seg_a)
num_seg_b = len(input_ids) - num_seg_a
print("Number of tokens in segment B (answer): ", num_seg_b)

SEP token index:  10
Number of tokens in segment A (question):  11
Number of tokens in segment B (answer):  460


In [17]:
#creating the segment ids and making sure every input token has a segment id
segment_ids = [0]*num_seg_a + [1]*num_seg_b
assert len(segment_ids) == len(input_ids)

Now the tokens and the segment ids will be passed inside the model

In [18]:
#token input_ids to represent the input and token segment_ids to differentiate our segments - question and text
output = model(torch.tensor([input_ids]),  token_type_ids = torch.tensor([segment_ids]))

Getting the start and end tokens from the output

In [19]:
#tokens with highest start and end scores
answer_start = torch.argmax(output.start_logits)
answer_end = torch.argmax(output.end_logits)
if answer_end >= answer_start:
    answer = " ".join(tokens[answer_start:answer_end+1])
else:
    print("I am unable to find the answer to this question. Can you please ask another question?")

print("Text:\n{}".format(new_df["text"][random_num]))
print("\nQuestion:\n{}".format(question.capitalize()))
print("\nAnswer:\n{}.".format(answer.capitalize()))

Text:
Homosexual couples in New York are being married for the first time after Governor Andrew Cuomo signed a law permitting same-sex marriage in the state last month. But such marriages, though now legal in New York, remain controversial . So many couples wanted to be married on Sunday, the first day of legal homosexual marriage in New York City, that local officials announced a lottery to determine qualification. New York City Council Speaker Christine Quinn, herself openly gay, witnessed the first two ceremonies of same-sex marriage. She says the same-sex marriage law supports equality. "A law that says every family is as good as every other family; that every family is based on love, and is exactly the way God wants it to be," she said. Margie Phelps announced same-sex marriage as a crime against God. She and other Christians from the Westboro Baptist Church in Topeka, Kansas came halfway across the country to say marriage is defined by Holy Bible. "The pattern is one man and one 

In [20]:
# cleaning the answer
answer = tokens[answer_start]
for i in range(answer_start+1, answer_end+1):
    if tokens[i][0:2] == "##":
        answer += tokens[i][2:]
    else:
        answer += " " + tokens[i]

In [21]:
print("\nAnswer:\n{}.".format(answer.capitalize()))


Answer:
Equality.


In [22]:
answer = new_df["answer"][random_num]
answer

'equality.'

Here cleaning is required when there are multiple tokens for a word. the double hash tells us that this is the same word split into multiple tokens

PART 2

In [23]:
# importing required libraries
import requests
import json
import torch
import os
from tqdm import tqdm
from transformers import BertTokenizerFast
from torch.utils.data import DataLoader
from transformers import BertForQuestionAnswering
from transformers import AdamW

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [24]:
# creating a directory in the google drive
if not os.path.exists('/content/drive/MyDrive/BERT-SQuAD'): 
  os.mkdir('/content/drive/MyDrive/BERT-SQuAD')

In [25]:
# getting the dataset
!wget -nc https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
!wget -nc https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json

--2022-11-11 19:48:21--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.111.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.111.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42123633 (40M) [application/json]
Saving to: ‘train-v2.0.json’

train-v2.0.json     100%[===================>]  40.17M  --.-KB/s    in 0.1s    

2022-11-11 19:48:22 (379 MB/s) - ‘train-v2.0.json’ saved [42123633/42123633]

--2022-11-11 19:48:22--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.111.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.111.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4370528 (4.2M) [application/json]
Saving to: ‘dev-v2.0.json’

dev-v2.0.json       1

In [26]:
# Load the training dataset and take a look at it
with open('train-v2.0.json', 'rb') as f:
  squad = json.load(f)

In [27]:
# Each 'data' dict has two keys (title and paragraphs)
squad['data'][0].keys()

dict_keys(['title', 'paragraphs'])

In [28]:
#displaying the data 
squad['data'][0]

{'title': 'Beyoncé',
 'paragraphs': [{'qas': [{'question': 'When did Beyonce start becoming popular?',
     'id': '56be85543aeaaa14008c9063',
     'answers': [{'text': 'in the late 1990s', 'answer_start': 269}],
     'is_impossible': False},
    {'question': 'What areas did Beyonce compete in when she was growing up?',
     'id': '56be85543aeaaa14008c9065',
     'answers': [{'text': 'singing and dancing', 'answer_start': 207}],
     'is_impossible': False},
    {'question': "When did Beyonce leave Destiny's Child and become a solo singer?",
     'id': '56be85543aeaaa14008c9066',
     'answers': [{'text': '2003', 'answer_start': 526}],
     'is_impossible': False},
    {'question': 'In what city and state did Beyonce  grow up? ',
     'id': '56bf6b0f3aeaaa14008c9601',
     'answers': [{'text': 'Houston, Texas', 'answer_start': 166}],
     'is_impossible': False},
    {'question': 'In which decade did Beyonce become famous?',
     'id': '56bf6b0f3aeaaa14008c9602',
     'answers': [{'text

Here we see that for each topic there are multiple paragraphs, and for each paragraph there are mutliple question and answer pairs

In [29]:
# checking the number of topics
len(squad['data'])

442

In [30]:
# loading the data in triplets of context, questions and answers
def read_data(path):  
  
  with open(path, 'rb') as f:
    squad = json.load(f)

  contexts = []
  questions = []
  answers = []

  for group in squad['data']:
    for passage in group['paragraphs']:
      context = passage['context']
      for qa in passage['qas']:
        question = qa['question']
        for answer in qa['answers']:
          contexts.append(context)
          questions.append(question)
          answers.append(answer)

  return contexts, questions, answers

In [31]:
#Splitting data into train and validation set 
train_contexts, train_questions, train_answers = read_data('train-v2.0.json')
valid_contexts, valid_questions, valid_answers = read_data('dev-v2.0.json')

In [32]:
#displaying the length of train and validation set 
print(f'There are {len(train_questions)} train questions')
print(f'There are {len(valid_questions)} dev questions')

There are 86821 train questions
There are 20302 dev questions


In [33]:
# cleaning the data
def add_end_idx(answers, contexts):
  for answer, context in zip(answers, contexts):
    gold_text = answer['text']
    start_idx = answer['answer_start']
    end_idx = start_idx + len(gold_text)

    # sometimes squad answers are off by a character or two so we fix this
    if context[start_idx:end_idx] == gold_text:
      answer['answer_end'] = end_idx
    elif context[start_idx-1:end_idx-1] == gold_text:
      answer['answer_start'] = start_idx - 1
      answer['answer_end'] = end_idx - 1     # When the gold label is off by one character
    elif context[start_idx-2:end_idx-2] == gold_text:
      answer['answer_start'] = start_idx - 2
      answer['answer_end'] = end_idx - 2     # When the gold label is off by two characters

add_end_idx(train_answers[:1000], train_contexts[:1000])
add_end_idx(valid_answers[:100], valid_contexts[:100])

In [34]:
# getting the model and its tokenizer (currently training on only 1000 rows as it is very time consuming)

tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

train_encodings = tokenizer(train_contexts[:1000], train_questions[:1000], truncation=True, padding=True)
valid_encodings = tokenizer(valid_contexts[:100], valid_questions[:100], truncation=True, padding=True)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [35]:
train_encodings.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
#Visualing the output of tokenizer, input ids are the token indices with padding of 0s, token_type_ids are different integers for different sequences and attention mask states 
#which positions to give attention to while training
train_encodings["input_ids"][0]

In [ ]:
valid_encodings["input_ids"][0]

In [ ]:
train_encodings["token_type_ids"][0]

In [ ]:
valid_encodings["token_type_ids"][0]

In [ ]:
train_encodings["attention_mask"][0]

In [39]:
# printing the number of training data
no_of_encodings = len(train_encodings['input_ids'])
print(f'We have {no_of_encodings} context-question pairs')

We have 1000 context-question pairs


In [40]:
# adding the answers in the training set for fine tuning
def add_token_positions(encodings, answers):
  start_positions = []
  end_positions = []
  for i in range(len(answers)):
    start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
    end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'] - 1))

    # if start position is None, the answer passage has been truncated
    if start_positions[-1] is None:
      start_positions[-1] = tokenizer.model_max_length
    if end_positions[-1] is None:
      end_positions[-1] = tokenizer.model_max_length

  encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

add_token_positions(train_encodings, train_answers[:1000])
add_token_positions(valid_encodings, valid_answers[:100])

In [41]:
# creating the dataset for BERT
class SQuAD_Dataset(torch.utils.data.Dataset):
  def __init__(self, encodings):
    self.encodings = encodings
  def __getitem__(self, idx):
    return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
  def __len__(self):
    return len(self.encodings.input_ids)

In [42]:
#Splitting the train and validation datatset
train_dataset = SQuAD_Dataset(train_encodings)
valid_dataset = SQuAD_Dataset(valid_encodings)

In [43]:
# Define the dataloaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=8)

In [44]:
# loading the BERT model which we will fine tune
model = BertForQuestionAnswering.from_pretrained("bert-base-uncased")

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased a

In [45]:
# checking the device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f'Working on {device}')

Working on cuda


In [46]:
# Fine tuning it per batch and defining the number of epochs 
N_EPOCHS = 5
optim = AdamW(model.parameters(), lr=5e-5)

model.to(device)
model.train()

for epoch in range(N_EPOCHS):
  loop = tqdm(train_loader, leave=True)
  for batch in loop:
    optim.zero_grad()
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    start_positions = batch['start_positions'].to(device)
    end_positions = batch['end_positions'].to(device)
    outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
    loss = outputs[0]
    loss.backward()
    optim.step()

    loop.set_description(f'Epoch {epoch+1}')
    loop.set_postfix(loss=loss.item())

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
Epoch 5: 100%|██████████| 125/125 [01:36<00:00,  1.29it/s, loss=0.178]


In [47]:
#performace evaluation
model.eval()

acc = []

for batch in tqdm(valid_loader):
  with torch.no_grad():
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    start_true = batch['start_positions'].to(device)
    end_true = batch['end_positions'].to(device)
    
    outputs = model(input_ids, attention_mask=attention_mask)

    start_pred = torch.argmax(outputs['start_logits'], dim=1)
    end_pred = torch.argmax(outputs['end_logits'], dim=1)

    acc.append(((start_pred == start_true).sum()/len(start_pred)).item())
    acc.append(((end_pred == end_true).sum()/len(end_pred)).item())

acc = sum(acc)/len(acc)

100%|██████████| 13/13 [00:02<00:00,  4.49it/s]


In [130]:
acc
print("Accuracy of model:",acc)

Accuracy of model: 0.07211538461538461


Part 2: Fine tune the BERT for question answering model on coqa dataset using similar process as shown in the demo for the squad dataset

In [49]:
# loading the dataset
coqa = pd.read_json('http://downloads.cs.stanford.edu/nlp/data/coqa/coqa-train-v1.0.json')
coqa

,version,data
0,1,"{'source': 'wikipedia', 'id': '3zotghdk5ibi9ce..."
1,1,"{'source': 'cnn', 'id': '3wj1oxy92agboo5nlq4r7..."
2,1,"{'source': 'gutenberg', 'id': '3bdcf01ogxu7zdn..."
3,1,"{'source': 'cnn', 'id': '3ewijtffvo7wwchw6rtya..."
4,1,"{'source': 'gutenberg', 'id': '3urfvvm165iantk..."
...,...,...
7194,1,"{'source': 'gutenberg', 'id': '34j10vatjfyw0ao..."
7195,1,"{'source': 'cnn', 'id': '3vj40nv2qinjocrcy7k4z..."
7196,1,"{'source': 'race', 'id': '3rjsc4xj10uw0to3vq0v..."
7197,1,"{'source': 'wikipedia', 'id': '3gs6s824sqxty8v..."


In [50]:
# dropping columns
del coqa["version"]

In [51]:
# structuring the dataset in usable format (creating one question and answer pair per row)
# by doing this, the text column will have repeated content until all questions of that paragraph are not covered
cols = ["text","question","answer", 'answer_start','answer_end']
comp_list = []
for index, row in coqa.iterrows():
    for i in range(len(row["data"]["questions"])):
        if row["data"]["answers"][i]["input_text"] == 'unknown':
          continue
        temp_list = []
        temp_list.append(row["data"]["story"])
        temp_list.append(row["data"]["questions"][i]["input_text"])
        temp_list.append(row["data"]["answers"][i]["input_text"])
        temp_list.append(row["data"]["answers"][i]['span_start'])
        temp_list.append(row["data"]["answers"][i]['span_end'])
        comp_list.append(temp_list)
new_df = pd.DataFrame(comp_list, columns = cols) 
new_df

,text,question,answer,answer_start,answer_end
0,"The Vatican Apostolic Library (), more commonl...",When was the Vat formally opened?,It was formally established in 1475,151,179
1,"The Vatican Apostolic Library (), more commonl...",what is the library for?,research,454,494
2,"The Vatican Apostolic Library (), more commonl...",for what subjects?,"history, and law",457,511
3,"The Vatican Apostolic Library (), more commonl...",and?,"philosophy, science and theology",457,545
4,"The Vatican Apostolic Library (), more commonl...",what was started in 2014?,a project,769,879
...,...,...,...,...,...
107271,(CNN) -- Cristiano Ronaldo provided the perfec...,Who was a sub?,Xabi Alonso,1405,1427
107272,(CNN) -- Cristiano Ronaldo provided the perfec...,Was it his first game this year?,Yes,1415,1467
107273,(CNN) -- Cristiano Ronaldo provided the perfec...,What position did the team reach?,third,1520,1555
107274,(CNN) -- Cristiano Ronaldo provided the perfec...,Who was ahead of them?,Barca.,1557,1582


In [52]:
#train test validation split
train_ratio = 0.70
validation_ratio = 0.15
test_ratio = 0.15

feature_columns = ['text','question']
dependent_columns = ['answer', 'answer_start', 'answer_end']

In [53]:
x_train, x_test, y_train, y_test = train_test_split(new_df[feature_columns], new_df[dependent_columns], test_size=1 - train_ratio)

In [54]:
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio)) 

In [55]:
if not os.path.exists('/content/drive/MyDrive/BERT-COQA'): 
  os.mkdir('/content/drive/MyDrive/BERT-COQA')

In [56]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')



In [57]:
#Defining train and validation encodings 
train_encodings = tokenizer(x_train['text'][:1000].values.tolist(), x_train['question'][:1000].values.tolist(), truncation=True, padding=True)
valid_encodings = tokenizer(x_val['text'][:100].values.tolist(), x_val['question'][:100].values.tolist(), truncation=True, padding=True)

In [58]:
train_encodings.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [59]:
# printing the number of training data
no_of_encodings = len(train_encodings['input_ids'])
print(f'We have {no_of_encodings} context-question pairs')

We have 1000 context-question pairs


In [131]:
#modifying answers for Fine tuning BERT
def add_token_positions1(encodings, answers):
  start_positions = []
  end_positions = []
  print(len(answers))
  for i in range(len(answers)):
    start_positions.append(encodings.char_to_token(i, answers.iloc[i]['answer_start']))
    end_positions.append(encodings.char_to_token(i, answers.iloc[i]['answer_end'] - 1))

    if start_positions[-1] is None:
      start_positions[-1] = tokenizer.model_max_length
    if end_positions[-1] is None:
      end_positions[-1] = tokenizer.model_max_length

  encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

In [61]:
add_token_positions1(train_encodings, y_train[:1000])
add_token_positions1(valid_encodings, y_val[:100])

1000
100


In [62]:
class coqa_Dataset(torch.utils.data.Dataset):
  def __init__(self, encodings):
    self.encodings = encodings
  def __getitem__(self, idx):
    return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
  def __len__(self):
    return len(self.encodings.input_ids)

In [63]:
#Splitting the dataset
train_dataset = coqa_Dataset(train_encodings)
valid_dataset = coqa_Dataset(valid_encodings)

In [64]:
# Define the dataloaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=8)

In [65]:
# loading the BERT model which we will fine tune
model = BertForQuestionAnswering.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased a

In [66]:
#device check
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f'Working on {device}')

Working on cuda


In [ ]:
# Fine tuning BERT
N_EPOCHS = 2
optim = AdamW(model.parameters(), lr=5e-5)
model.to(device)
model.train()

In [68]:
for epoch in range(N_EPOCHS):
  loop = tqdm(train_loader, leave=True)
  for batch in loop:
    optim.zero_grad()
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    start_positions = batch['start_positions'].to(device)
    end_positions = batch['end_positions'].to(device)
    outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
    loss = outputs[0]
    loss.backward()
    optim.step()

    loop.set_description(f'Epoch {epoch+1}')
    loop.set_postfix(loss=loss.item())

Epoch 2: 100%|██████████| 125/125 [01:46<00:00,  1.17it/s, loss=3.37]


In [69]:
#Performance evaluation 
model.eval()

acc = []

for batch in tqdm(valid_loader):
  with torch.no_grad():
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    start_true = batch['start_positions'].to(device)
    end_true = batch['end_positions'].to(device)
    
    outputs = model(input_ids, attention_mask=attention_mask)

    start_pred = torch.argmax(outputs['start_logits'], dim=1)
    end_pred = torch.argmax(outputs['end_logits'], dim=1)

    acc.append(((start_pred == start_true).sum()/len(start_pred)).item())
    acc.append(((end_pred == end_true).sum()/len(end_pred)).item())

acc = sum(acc)/len(acc)

100%|██████████| 13/13 [00:03<00:00,  3.48it/s]


In [132]:
acc
print("Accuracy of model:",acc)

Accuracy of model: 0.07211538461538461


Part 3: Import BERT for classification model which is already fine tuned and apply it on any text classification dataset such as the twitter dataset

In [71]:
#Download and load twitter dataset 

train_data = load_dataset('tweet_eval', 'emotion', split='train[:1000]')
val_data = load_dataset('tweet_eval', 'emotion', split='validation[:100]')

Extracting data files:   0%|          | 0/6 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/3257 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1421 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/374 [00:00<?, ? examples/s]

Dataset tweet_eval downloaded and prepared to /root/.cache/huggingface/datasets/tweet_eval/emotion/1.1.0/12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343. Subsequent calls will reuse this data.


In [133]:
train_data

Dataset({
    features: ['text', 'label'],
    num_rows: 1000
})

In [134]:
val_data

Dataset({
    features: ['text', 'label'],
    num_rows: 100
})

In [72]:

label_dict= {0:'anger', 
             1:'joy',
             2:'optimism',
             3:'sadness'}

In [73]:
lst =[]

In [74]:
lst.append(train_data['text'][1])
lst

["My roommate: it's okay that we can't spell because we have autocorrect. #terrible #firstworldprobs"]

In [75]:
#Pretrained model 

model_id = 'philschmid/BERT-tweet-eval-emotion'

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSequenceClassification.from_pretrained(model_id)

classifier = pipeline('text-classification', tokenizer=tokenizer, model=model)
# classifier("Worry is a down payment on a problem you may never have'. Joyce Meyer.  #motivation #leadership #worry")


Downloading:   0%|          | 0.00/312 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

In [76]:
 #tokeninzing process for the input text
random_num = np.random.randint(0,len(train_data))
curr_lst = []
curr_lst.append(train_data['text'][random_num])
curr_label = train_data['label'][random_num]


In [77]:
# encoding with max length = 512
input_ids = tokenizer.encode(curr_lst,is_split_into_words=True, max_length=512,truncation=True)
print("The input has a total of {} tokens.".format(len(input_ids)))

The input has a total of 19 tokens.


In [78]:
# visualizing the tokens
tokens = tokenizer.convert_ids_to_tokens(input_ids)
for token, id in zip(tokens, input_ids):
    print('{:8}{:8,}'.format(token,id))

[CLS]        101
@            137
user       4,795
made       1,189
me         1,143
laugh      4,046
,            117
that       1,115
quote     15,882
.            119
In         1,130
a            170
sort       3,271
of         1,104
rue       27,131
##ful      2,365
way        1,236
.            119
[SEP]        102


In [87]:
# Visualizing the number of token in text
sep_token_idx = input_ids.index(tokenizer.sep_token_id)
print("SEP token index: ", sep_token_idx)


SEP token index:  18


In [92]:
num_seg_a = sep_idx + 1
print("Number of tokens in segment A (text): ", num_seg_a)

Number of tokens in segment A (text):  11


In [99]:
# Visualizing the number of token in question and text
sep_idx = input_ids.index(tokenizer.sep_token_id)
print("SEP token index: ", sep_idx)
num_seg_a = sep_idx + 1
print("Number of tokens in segment A (question): ", num_seg_a)
num_seg_b = len(input_ids) - num_seg_a
print("Number of tokens in segment B (answer): ", num_seg_b)

SEP token index:  18
Number of tokens in segment A (question):  19
Number of tokens in segment B (answer):  0


In [102]:
#creating the segment ids and making sure every input token has a segment id
segment_ids = [0]*num_seg_a + [1]*num_seg_b
assert len(segment_ids) == len(input_ids)

In [103]:
#creating the segment ids and making sure every input token has a segment id
segment_ids = [0]*num_seg_a 

In [104]:
len(segment_ids) == len(input_ids)

True

In [105]:
len(segment_ids)

19

In [106]:
output = model(torch.tensor([input_ids]),  token_type_ids = torch.tensor([segment_ids]))

In [107]:
output

SequenceClassifierOutput(loss=None, logits=tensor([[-1.1318,  1.9999, -0.7659,  0.1579]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [108]:
res = int(torch.argmax(output.logits))

In [109]:
label_dict[res]

'joy'

In [110]:
label_dict[curr_label]

'joy'

Part 4: Fine tune the BERT for classification model on the text classification dataset you used in the previous question and check the performance

In [111]:
train_data = load_dataset('tweet_eval', 'emotion', split='train[:1000]')
val_data = load_dataset('tweet_eval', 'emotion', split='validation[:100]')

In [112]:
# create a map of labels
label_map= {0:'anger',
             1:'joy',
             2:'optimism',
             3:'sadness'}

In [113]:
# using pretrained bert-base-uncased
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased',do_lower_case=True)

In [114]:
train_encodings = tokenizer(train_data['text'], truncation=True, padding=True,max_length=512)


In [115]:
valid_encodings = tokenizer(val_data['text'], truncation=True, padding=True,max_length=512)

In [116]:

class TwitterDataset(torch.utils.data.Dataset):
    """Class to get length and labels of the tweets"""
    
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)

In [117]:
en_train_dataset = TwitterDataset(train_encodings, train_data['label'])
en_valid_dataset = TwitterDataset(valid_encodings, val_data['label'])

In [118]:
en_train_dataset

In [119]:
# printing the number of training data
no_of_encodings = len(train_encodings['input_ids'])
print(f'We have {no_of_encodings} examples')

We have 1000 examples


In [120]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_map))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [121]:
# Define the dataloaders
train_loader = DataLoader(en_train_dataset, batch_size=8, shuffle=True)


In [122]:
valid_loader = DataLoader(en_valid_dataset, batch_size=8)

In [123]:
# using adam optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)
# no of epochs
num_epochs = 2
num_training_steps = num_epochs * len(train_loader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [124]:
# Fine tuning it per batch
progress_bar = tqdm(range(num_training_steps))
# train the model with fine tuning
model.train()
for epoch in range(num_epochs):
    for batch in train_loader:
        # batch size
        batch = {k: v.to(device) for k, v in batch.items()}
        # to use it on same device added to(device) call
        outputs = model.to(device)(**batch)
        # calculate the loss
        loss = outputs.loss
        # backward propagation
        loss.backward()
        # optimizer
        optimizer.step()
        lr_scheduler.step()
        
        optimizer.zero_grad()
        # show progress
        progress_bar.update(1)

100%|██████████| 250/250 [00:29<00:00,  8.50it/s]

In [125]:
# 
metric = evaluate.load("accuracy")
model.eval()
for batch in valid_loader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

{'accuracy': 0.8}

References :
1. https://www.analyticsvidhya.com/blog/2020/07/transfer-learning-for-nlp-fine-tuning-bert-for-text-classification/
2. https://www.geeksforgeeks.org/fine-tuning-bert-model-for-sentiment-analysis/
3. https://towardsdatascience.com/fine-tuning-bert-for-text-classification-54e7df642894

